# Exercise 8-1: Analyze the Forest Fires data

**Name:**  Leah Nicholson <br/>
**Class:**  Data Visualization DEV 228<br/>
**Date:**  05/31/2025<br/>
**Desc:**  Analyzing data from forest fires file, grouping and aggregating, using pivot tables and bins. <br/>

## Read the data

In [4]:
import pandas as pd

In [5]:
fires_by_month = pd.read_pickle('fires_by_month.pkl')

In [6]:
fires_by_month = fires_by_month.sort_values('fire_year')    # sorting

In [7]:
# display the first five rows
fires_by_month.head()    # earliest year is 1992

,state,fire_year,fire_month,acres_burned,days_burning,fire_count
0,AK,1992,5,4202.0,135.0,14
2084,HI,1992,7,1583.0,97.0,1
2085,HI,1992,8,248.0,0.0,0
2086,HI,1992,9,40.0,0.0,0
2087,HI,1992,10,1566.0,90.0,1


In [8]:
fires_by_month.tail()     # latest year is 2015

,state,fire_year,fire_month,acres_burned,days_burning,fire_count
8065,TX,2015,4,2118.00,0.0,20
8066,TX,2015,6,2899.90,0.0,19
8067,TX,2015,7,9658.26,18.0,98
673,AR,2015,11,225.00,3.0,2
9299,WY,2015,12,72.00,0.0,2


## Group and aggregate the data

In [10]:
# group the data by state and year
fires_grouped = fires_by_month.groupby(['state', 'fire_year'], observed = False)

In [11]:
# sum the grouped data and store DataFrame in fires_by_year variable
fires_by_year = fires_grouped.sum()

# After grouping, applying .sum() aggregates all numerical columns within each group (state and fire_year) by summing their values. 
# This effectively gives the total acres_burned, days_burning, and fire_count for each state in each year.

In [12]:
# drop the fire_month column
fires_by_year.drop(columns = ['fire_month'], inplace = True)    # inplace will keep the changes in fires_by_year DataFrame

In [13]:
# A check:
fires_by_year.head()    # data grouped with a multi-index, and fire_month dropped

acres_burned  days_burning  fire_count
state fire_year                                        
AK    1992           142444.7        1145.0          68
      1993           686630.5        3373.0         144
      1994           261604.7        2517.0         126
      1995            43762.6         880.0          48
      1996           598407.2        2112.0          88

## Use pivot tables

In [15]:
# select rows where the year is 2013 or later
# Use the query() method to select all the data for the years 2013 and later. Then, reset the index for the DataFrame that’s returned and assign the DataFrame to a variable named fires_recent. 

fires_recent = fires_by_year.query('fire_year >= 2013')   # selects all rows where fire_year is 2013 or greater
fires_recent = fires_recent.reset_index()    # fire_year was part of multi-index; this puts state and fire_year back into regular columns

# Check:
fires_recent.head()

,state,fire_year,acres_burned,days_burning,fire_count
0,AK,2013,1320518.50,4644.0,153
1,AK,2014,233426.40,571.0,43
2,AK,2015,5136372.99,16636.0,340
3,AL,2013,28015.65,115.0,581
4,AL,2014,35310.20,107.0,758


In [16]:
# Use the pivot() method to pivot the data so the STATE column provides the values for the ROW labels, 
# the FIRE_YEAR column provides the values for the COLUMN labels, 
# and the ACRES_BURNED column provides the data (VALUES) for the table (used to populate the new DataFrame). 

fires_pivot_version = fires_recent.pivot(index = 'state', columns = 'fire_year', values = 'acres_burned')

# Check:
fires_pivot_version.head()

fire_year,2013,2014,2015
state,,,
AK,1320518.50,233426.40,5136372.99
AL,28015.65,35310.20,28386.96
AR,11797.60,18521.80,16302.96
AZ,102307.47,202212.51,154720.38
CA,575555.65,543033.33,844674.02


In [17]:
# Using the pivot_table() method with the fires_by_month DataFrame to get the same result as the previous step. I note this saves several steps.

# aggfunc specifies an aggregate method (or list of methods) to be applied to each column specified by the values parameter
# pivot() disallows duplicates and does NOT perform aggregation, only reshaping
# So, since pivot() worked, this should also work, since this one allows duplicates, but pivot_table does not:

fires_pivot_table_version = fires_recent.pivot_table(index = 'state', columns = 'fire_year', values = 'acres_burned', aggfunc = 'sum', observed = False)   # observed = False retains all columns, even if they have only NaN
fires_pivot_table_version.head()

# Confirmed:  same output as above

fire_year,2013,2014,2015
state,,,
AK,1320518.50,233426.40,5136372.99
AL,28015.65,35310.20,28386.96
AR,11797.60,18521.80,16302.96
AZ,102307.47,202212.51,154720.38
CA,575555.65,543033.33,844674.02


## Work with bins

In [19]:
# Check:
fires_by_year.head()

acres_burned  days_burning  fire_count
state fire_year                                        
AK    1992           142444.7        1145.0          68
      1993           686630.5        3373.0         144
      1994           261604.7        2517.0         126
      1995            43762.6         880.0          48
      1996           598407.2        2112.0          88

In [20]:
# reset the index for the DataFrame named fires_by_year - I'm also including a sort by year
fires_by_year = fires_by_year.reset_index().sort_values('fire_year')

# Check: confirmed, index reset (former multi-index is now converted to columns, and there's a basic numerical index)
fires_by_year.head()

,state,fire_year,acres_burned,days_burning,fire_count
0,AK,1992,142444.70,1145.0,68
216,GA,1992,16892.16,60.0,6
480,ME,1992,3750.90,7.0,44
96,CA,1992,289254.90,434.0,819
1128,WA,1992,88514.00,135.0,60


In [21]:
# bin the data by decade - cut creates equally sized bins
fires_by_year['decade'] = pd.cut(fires_by_year.fire_year, bins = [0, 1999, 2009, 2019], labels = ['1990s', '2000s', '2010s'])

In [22]:
# double-check the edge values: MIN
min_year = fires_by_year.fire_year.min()
min_year   # fire_year min should be 1992 - checks out

1992

In [23]:
# double-check the edge values: MAX
max_year = fires_by_year.fire_year.max()
max_year   # fire_year max should be 2015 - checks out

2015

In [24]:
fires_by_year.head(25)

,state,fire_year,acres_burned,days_burning,fire_count,decade
0,AK,1992,142444.70,1145.0,68,1990s
216,GA,1992,16892.16,60.0,6,1990s
480,ME,1992,3750.90,7.0,44,1990s
96,CA,1992,289254.90,434.0,819,1990s
1128,WA,1992,88514.00,135.0,60,1990s
696,NH,1992,15.00,0.0,0,1990s
336,IN,1992,160.60,0.0,6,1990s
864,OR,1992,115862.55,467.0,146,1990s
720,NJ,1992,15551.35,0.0,5,1990s
504,MI,1992,2162.70,140.0,11,1990s


In [25]:
# Extra check:  last (max) value for fire_year - confirmed
fires_by_year.tail()

,state,fire_year,acres_burned,days_burning,fire_count,decade
23,AK,2015,5136372.99,16636.0,340,2010s
1007,SD,2015,52758.22,34.0,166,2010s
191,DE,2015,56.00,1.0,3,2010s
215,FL,2015,162845.80,1099.0,554,2010s
1223,WY,2015,36233.50,351.0,107,2010s


In [26]:
# drop the fire_year column and assign the DataFrame that's returned to a variable named fires_by_decade
fires_by_decade = fires_by_year.drop('fire_year', axis = 1)
fires_by_decade

,state,acres_burned,days_burning,fire_count,decade
0,AK,142444.70,1145.0,68,1990s
216,GA,16892.16,60.0,6,1990s
480,ME,3750.90,7.0,44,1990s
96,CA,289254.90,434.0,819,1990s
1128,WA,88514.00,135.0,60,1990s
...,...,...,...,...,...
23,AK,5136372.99,16636.0,340,2010s
1007,SD,52758.22,34.0,166,2010s
191,DE,56.00,1.0,3,2010s
215,FL,162845.80,1099.0,554,2010s


In [27]:
# group by the state and decade columns, and sum the data
fires_by_decade.groupby(by = ['state', 'decade'], observed = False)[['acres_burned', 'days_burning', 'fire_count']].sum().round(2)

# This sums by state/decade each of acres_burned, days_burning, fire_count for totals for each state/decade

acres_burned  days_burning  fire_count
state decade                                        
AK    1990s     4884868.50       15069.0         734
      2000s    18920624.10       34174.0         974
      2010s     8420517.59       31025.0         915
AL    1990s      194592.80         214.0         216
      2000s      408880.50        1863.0         535
...                    ...           ...         ...
WV    2000s      230848.90         804.0        1418
      2010s       84069.07         393.0         724
WY    1990s      424074.40         675.0         450
      2000s     1166739.81        7382.0         720
      2010s      799344.94        6990.0         687

[153 rows x 3 columns]